In [1]:
import pandas as pd
import psycopg2

In [2]:
df_banks = pd.read_csv("../data/banks.csv")
print(df_banks.head())
df_reviews = pd.read_csv("../data/processed/reviews_processed.csv")
print(df_reviews.head())

   bank_id                    bank_name
0        1  Commercial Bank of Ethiopia
1        2                   Awash Bank
2        3                  Dashen Bank
                              review_id  \
0  ba01d6b7-97fe-4376-b68c-817f397c00af   
1  bde60e8b-0b05-4c88-9f60-c14c18fa57b9   
2  fae55af8-1c5a-41f5-b6e8-55652afbf303   
3  e35ece9f-a9fb-4510-a9e3-8b8487a99d12   
4  fb9ed669-ea1d-4eb5-a336-634ccc59e265   

                                         review_text  rating review_date  \
0  I can't access the loan in the app. only it wo...       3  2025-11-28   
1     no one from all ethiopian bank keep it up nice       5  2025-11-28   
2                                      thenk you ✍️🙏       5  2025-11-26   
3                              i hope to be good app       5  2025-11-24   
4                                         amazing 😍😍       5  2025-11-23   

   review_year  review_month bank_code   bank_name           user_name  \
0         2025            11     Awash  Awash Bank

In [7]:
# -------------------------------------------------------
# Connect to PostgreSQL database
# -------------------------------------------------------


conn = psycopg2.connect(
    host="localhost",              
    database="bank_reviews", 
    user="admin",                 
    password="admin123"            
)

# Create a cursor object — used to execute SQL commands
cur = conn.cursor()

In [25]:
conn.rollback()


In [13]:
from psycopg2.extras import execute_values

rows = []
for row in df_banks.itertuples(index=False):
    bank_id = int(row.bank_id)
    bank_name = None if pd.isna(row.bank_name) else row.bank_name
    rows.append((bank_id, bank_name))

sql = """
INSERT INTO banks (bank_id, bank_name)
VALUES %s
ON CONFLICT (bank_id) DO NOTHING;
"""
execute_values(cur, sql, rows)
conn.commit()

In [16]:
for row in df_reviews.itertuples(index=False):
    print(row)  # inspect to see the attribute names


Pandas(review_id='ba01d6b7-97fe-4376-b68c-817f397c00af', review_text="I can't access the loan in the app. only it work in USSD", rating=3, review_date='2025-11-28', review_year=2025, review_month=11, bank_code='Awash', bank_name='Awash Bank', user_name='FAHMI SHEMSEDIN', thumbs_up=0, text_length=56, source='Google Play')
Pandas(review_id='bde60e8b-0b05-4c88-9f60-c14c18fa57b9', review_text='no one from all ethiopian bank keep it up nice', rating=5, review_date='2025-11-28', review_year=2025, review_month=11, bank_code='Awash', bank_name='Awash Bank', user_name='Zed Cherinet', thumbs_up=0, text_length=46, source='Google Play')
Pandas(review_id='fae55af8-1c5a-41f5-b6e8-55652afbf303', review_text='thenk you ✍️🙏', rating=5, review_date='2025-11-26', review_year=2025, review_month=11, bank_code='Awash', bank_name='Awash Bank', user_name='dechasa leta', thumbs_up=0, text_length=13, source='Google Play')
Pandas(review_id='e35ece9f-a9fb-4510-a9e3-8b8487a99d12', review_text='i hope to be good ap

In [17]:
# bank_name -> bank_id
bank_mapping = dict(zip(df_banks['bank_name'], df_banks['bank_id']))

In [21]:
rows = []
for row in df_reviews.itertuples(index=False):
    review_id = row.review_id  # UUID string
    bank_id = bank_mapping.get(row.bank_name)  # map bank_name to bank_id

    if bank_id is None:
        continue

    review_text = None if pd.isna(row.review_text) else row.review_text
    rating = float(row.rating) if not pd.isna(row.rating) else None
    review_date = row.review_date
    sentiment_label = None
    sentiment_score = None
    source = None if pd.isna(row.source) else row.source

    rows.append((review_id, bank_id, review_text, rating, review_date,
                 sentiment_label, sentiment_score, source))


In [26]:
from psycopg2.extras import execute_values

sql = """
INSERT INTO reviews
    (review_id, bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, source)
VALUES %s
ON CONFLICT (review_id) DO NOTHING;
"""

execute_values(cur, sql, rows)
conn.commit()
cur.close()
conn.close()

print("Data inserted successfully!")


Data inserted successfully!
